# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [414]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [415]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [4]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhoY-gpAMFUAAE89mFcAAABA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [5]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [6]:
#content_analysis_save = 'wikipedia_content_analysis.html'

#writing the requested text into html file locally
with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [7]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [8]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   More generally, content analysis is research u...
7   By having contents of communication available ...
8   Robert Weber notes: "To make valid inferences ...
9   There are five types of texts in content analy...
10  Over the years, content analysis has been appl...
11  In recent times, particularly with the advent ...
12  Quantitative content analysis has enjoyed a re...
13  Content analysis can also be described as stud...
14  Holsti groups fifteen uses of content analysis...
15  He also places these uses into the context of ...
16  The following table shows fifteen uses of cont...
17  The process of the initi

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [18]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [20]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [22]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [24]:
# Request for content
page = requests.get("https://features.unicef.org/state-of-the-worlds-children-2019-nutrition/policy-agenda/index.html")
page

<Response [200]>

In [25]:
# Parse using bs4
page_soup = bs4.BeautifulSoup(page.text, 'html.parser')

In [88]:
# Find appropriate tags to get content
content = page_soup.findAll(['p','li'])
content

details = ''
flag=False
for para in content:
    if para.text=='':
        flag=True
    if flag:
        details += para.text
    if para.text=='CASE STUDY':
        flag=False

In [94]:
d_list = re.split(r"CASE STUDY", details)[:-1]

In [226]:
# Cleaning to extract only the details of each recommendations using regex
d_cleaned = []
for d in d_list:
    replace_list= re.findall(r'[a-z]\B[A-Z]|[:)][A-Z]', d)
    for item in replace_list:
        if item[0]==')':
            d = re.sub('\\'+item, item[0]+'. '+item[1], d)
        else:
            d = re.sub(item, item[0]+'. '+item[1], d)
    d = re.sub(":.",":",d)
    d = re.sub('\\xa0', '. ', d)
    d = re.sub(r'(?<=:)\b(?=[A-Z])', ' ', d)
    d = re.sub(r'(?<=.)(?<!\s)\b(?=[A-Z])', ' ', d)
    d += '.'

    d_cleaned.append(d)

In [223]:
# Extracting recommendations and case study titles
recommendations = page_soup.findAll('h2','Theme-Layer-BodyText-Heading-Large')

title = []
rec = []
for r in recommendations:
    if " in " in r.text:
        title.append(r.text)
    else:
        rec.append(re.sub('\\xa0', '', r.text))

In [224]:
# Extracting case study country and topic
topic = []
country = []
for t in title:
    print(t)
    tp, c = re.split(r" in ", t)
    topic.append(tp)
    country.append(c)

Sugar taxes in Mexico
Locally produced ready-to-use therapeutic food in Cambodia
Labelling in Chile
Integrating water, sanitation and nutrition interventions in Pakistan
Nutrition surveys in India


In [233]:
# Putting information together in a Dataframe.
df1 = pandas.DataFrame({'recommendations': rec, 'elaboration': d_cleaned, 'casestudy_country': country, 'casestudy_topic': topic})
df1

,recommendations,elaboration,casestudy_country,casestudy_topic
0,"Empower families, children and young people to...",Increasing demand for healthy foods is an impo...,Mexico,Sugar taxes
1,Drive food suppliers to do the right thing for...,It’s not enough that children and families dem...,Cambodia,Locally produced ready-to-use therapeutic food
2,Build healthy food environments for all children,Children and families shouldn’t have to face b...,Chile,Labelling
3,Mobilize supportive systems to scale up nutrit...,A child’s nutrition level is influenced by mor...,Pakistan,"Integrating water, sanitation and nutrition in..."
4,"Collect, analyse and use good-quality data and...","We need accurate, quality data not only to und...",India,Nutrition surveys



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [234]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [235]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,6
7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,8
9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [236]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [237]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [257]:
# function to get links
def get_links(list_of_tags):
    urls = []
    for num, tag in enumerate(list_of_tags):
        tag_links = tag.findAll('a', href=re.compile('talkpoverty.org'), class_=False)
        for link in tag_links:
            #We need to extract the url from the <a> tag
            url = link.get('href')
            link_text = link.text.strip()
            urls.append((
                url,
                num,
                link_text,
            ))
    return urls

In [269]:
# get the links for the website
s_website="https://talkpoverty.org/category/feature/"
page = requests.get(s_website)
page_soup = bs4.BeautifulSoup(page.text, 'html.parser')
content = page_soup.findAll('h2')
urls = get_links(content)

In [270]:
text = []
for para in content:
    text.append(para.text.strip())
text=text[:-1]

In [282]:
df2 = pandas.DataFrame({'text': text})
df2['source'] = [s_website] * len(df2['text'])
df2['paragraph-number'] = range(len(df2['text']))
df2['source-paragraph-number'] = [None] * len(df2['text'])
df2['source-paragraph-text'] = [None] * len(df2['text'])
df2

,text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Environmental Racism Is Killing Black Communit...,https://talkpoverty.org/category/feature/,0,None,None
1,A Mistake At A Ticket Machine Cost Me $100. Fi...,https://talkpoverty.org/category/feature/,1,None,None
2,A Pesticide the EPA Won’t Ban Is Sickening Low...,https://talkpoverty.org/category/feature/,2,None,None
3,Major League Baseball Wants to Crush 42 Minor ...,https://talkpoverty.org/category/feature/,3,None,None
4,"A Unique Philadelphia Law Guarantees 16,000 Do...",https://talkpoverty.org/category/feature/,4,None,None
5,"For D.C. Parents, School Chaperoning Is Pay to...",https://talkpoverty.org/category/feature/,5,None,None
6,State Laws Can Punish Parents Living in Abusiv...,https://talkpoverty.org/category/feature/,6,None,None
7,Dangerous Jobs. Harassment. Long Hours. Welcom...,https://talkpoverty.org/category/feature/,7,None,None
8,A Census Undercount Likely Cost Detroit $1.3 M...,https://talkpoverty.org/category/feature/,8,None,None
9,Getting Time Off Work To Support Disabled Kids...,https://talkpoverty.org/category/feature/,9,None,None


In [285]:
def getTextFromTalkPov(targetURL, sourceParNum, sourceText):
    parsDict = {'source' : [], 'paragraph-number' : [], 'text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    urls = []
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #get text
        parsDict['text'].append(pTag.text)
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)


for urlTuple in urls:
    #ignore_index means the indices will not be reset after each append
    df2 = df2.append(getTextFromTalkPov(*urlTuple),ignore_index=True)

In [286]:
df2

,paragraph-number,source,source-paragraph-number,source-paragraph-text,text
0,0,https://talkpoverty.org/category/feature/,None,None,Environmental Racism Is Killing Black Communit...
1,1,https://talkpoverty.org/category/feature/,None,None,A Mistake At A Ticket Machine Cost Me $100. Fi...
2,2,https://talkpoverty.org/category/feature/,None,None,A Pesticide the EPA Won’t Ban Is Sickening Low...
3,3,https://talkpoverty.org/category/feature/,None,None,Major League Baseball Wants to Crush 42 Minor ...
4,4,https://talkpoverty.org/category/feature/,None,None,"A Unique Philadelphia Law Guarantees 16,000 Do..."
5,5,https://talkpoverty.org/category/feature/,None,None,"For D.C. Parents, School Chaperoning Is Pay to..."
6,6,https://talkpoverty.org/category/feature/,None,None,State Laws Can Punish Parents Living in Abusiv...
7,7,https://talkpoverty.org/category/feature/,None,None,Dangerous Jobs. Harassment. Long Hours. Welcom...
8,8,https://talkpoverty.org/category/feature/,None,None,A Census Undercount Likely Cost Detroit $1.3 M...
9,9,https://talkpoverty.org/category/feature/,None,None,Getting Time Off Work To Support Disabled Kids...


In [477]:
def recursive_spider(targetURL, sourceParNum, sourceText, limit,i=0):
    parsDict = {'source' : [], 'paragraph-number' : [], 'text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    urls = []

    #Now we get the page
    print('getting page {}'.format(i))
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):        
        # get para
        parsDict['text'].append(pTag.text)
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL) 
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
        
        #get links
        tag_links = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for link in tag_links:
            relurl = link.get('href')
            link_text = link.text.strip()
            urls.append((
                urllib.parse.urljoin(wikipedia_base_url, relurl),
                parNum,
                link_text))
    df = pandas.DataFrame(parsDict)
    
    print(urls[:limit])
    for urlTuple in urls[:limit]:
        if i>limit: 
            # limit the number of recursive depth
            break
        else: 
            i += 1
            #ignore_index means the indices will not be reset after each append
            df = df.append(recursive_spider(*urlTuple,limit,i),ignore_index=True)

    return df

In [478]:
website="https://en.wikipedia.org/wiki/Poverty"
df2a=recursive_spider(website, None, None, limit=2)

getting page 0
[('https://en.wikipedia.org/wiki/Income', 1, 'income'), ('https://en.wikipedia.org/wiki/Social', 1, 'social')]
getting page 1
[('https://en.wikipedia.org/wiki/Gross_income', 1, 'gross income'), ('https://en.wikipedia.org/wiki/Pension', 1, 'pension')]
getting page 2
[('https://en.wikipedia.org/wiki/Net_income', 0, 'net income'), ('https://en.wikipedia.org/wiki/Pension', 0, 'pension')]
getting page 3
[('https://en.wikipedia.org/wiki/Business', 0, 'business'), ('https://en.wikipedia.org/wiki/Income', 0, 'income')]
getting page 3
[('https://en.wikipedia.org/wiki/Help:IPA/English', 1, '/ˈpɛnʃən/'), ('https://en.wikipedia.org/wiki/Latin', 1, 'Latin')]
getting page 2
[('https://en.wikipedia.org/wiki/Organism', 0, 'organisms'), ('https://en.wikipedia.org/wiki/Human', 0, 'humans')]
getting page 3
[('https://en.wikipedia.org/wiki/Biology', 1, 'biology'), ('https://en.wikipedia.org/wiki/Ancient_Greek', 1, 'Greek')]


In [479]:
df2a['source'].unique()

array(['https://en.wikipedia.org/wiki/Poverty',
       'https://en.wikipedia.org/wiki/Income',
       'https://en.wikipedia.org/wiki/Gross_income',
       'https://en.wikipedia.org/wiki/Net_income',
       'https://en.wikipedia.org/wiki/Pension',
       'https://en.wikipedia.org/wiki/Social',
       'https://en.wikipedia.org/wiki/Organism'], dtype=object)

In [480]:
df2a

,source,paragraph-number,text,source-paragraph-number,source-paragraph-text
0,https://en.wikipedia.org/wiki/Poverty,0,\n,None,None
1,https://en.wikipedia.org/wiki/Poverty,1,Poverty is not having enough material possessi...,None,None
2,https://en.wikipedia.org/wiki/Poverty,2,Absolute poverty is the complete lack of the m...,None,None
3,https://en.wikipedia.org/wiki/Poverty,3,"On the other hand, relative poverty occurs whe...",None,None
4,https://en.wikipedia.org/wiki/Poverty,4,Governments and non-governmental organizations...,None,None
5,https://en.wikipedia.org/wiki/Poverty,5,"In 2012 it was estimated that, using a poverty...",None,None
6,https://en.wikipedia.org/wiki/Poverty,6,The World Bank forecasted in 2015 that 702.1 m...,None,None
7,https://en.wikipedia.org/wiki/Poverty,7,During the 2013 to 2015 period The World Bank ...,None,None
8,https://en.wikipedia.org/wiki/Poverty,8,There is disagreement among experts as to what...,None,None
9,https://en.wikipedia.org/wiki/Poverty,9,"An data based scientific empirical research, w...",None,None


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [287]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3463,"posts-type":false,"posts":[{"id":"190233110922","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922\/a-most-fortunate-red-headed-stepkitten","type":"photo","date-gmt":"2020-01-13 11:03:57 GMT","date":"Mon, 13 Jan 2020 12:03:57","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1578913437,"format":"html","reblog-key":"BRsWJpjQ","slug":"a-most-fortunate-red-headed-stepkitten","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"190233110922\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_190233110922\"><iframe id=\"like_iframe_190233110922\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [295]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

d

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
1


{'tumblelog': {'title': 'One hour one pic lolcats',
  'description': '',
  'name': 'lolcats-lol-cat',
  'timezone': 'Europe/Paris',
  'cname': False,
  'feeds': []},
 'posts-start': 0,
 'posts-total': 3463,
 'posts-type': False,
 'posts': [{'id': '146020177084',
   'url': 'https://lolcats-lol-cat.tumblr.com/post/146020177084',
   'url-with-slug': 'https://lolcats-lol-cat.tumblr.com/post/146020177084/cat-lolcat-lolcats',
   'type': 'photo',
   'date-gmt': '2016-06-16 18:00:47 GMT',
   'date': 'Thu, 16 Jun 2016 20:00:47',
   'bookmarklet': 0,
   'mobile': 0,
   'feed-item': '',
   'from-feed-id': 0,
   'unix-timestamp': 1466100047,
   'format': 'html',
   'reblog-key': 'aVnwKamC',
   'slug': 'cat-lolcat-lolcats',
   'is-submission': False,
   'like-button': '<div class="like_button" data-post-id="146020177084" data-blog-name="lolcats-lol-cat" id="like_button_146020177084"><iframe id="like_iframe_146020177084" src="https://assets.tumblr.com/assets/html/like_iframe.html?_v=66c22ab5319d742b

If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [293]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [294]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,190233110922,https://66.media.tumblr.com/42458742dedc510f3d...,"Mon, 13 Jan 2020 12:03:57","[cat, cats, lol, lolcat, lolcats]",jpg
1,190232075547,https://66.media.tumblr.com/65b1e6efd6ec145872...,"Mon, 13 Jan 2020 10:00:35","[cat, cats, lol, lolcat, lolcats]",jpg
2,190230902981,https://66.media.tumblr.com/616b4faab5a289a369...,"Mon, 13 Jan 2020 08:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
3,190229342226,https://66.media.tumblr.com/28121ad01b05305b89...,"Mon, 13 Jan 2020 06:00:57","[cat, cats, lol, lolcat, lolcats]",jpg
4,190209640041,https://66.media.tumblr.com/9b735740f517f3673b...,"Sun, 12 Jan 2020 06:01:05","[cat, cats, lol, lolcat, lolcats]",jpg
5,190193496849,https://66.media.tumblr.com/1ffa3c880a55f80f62...,"Sat, 11 Jan 2020 10:01:29","[cat, cats, lol, lolcat, lolcats]",jpg
6,190192270965,https://66.media.tumblr.com/a5ecfce3b06defdbac...,"Sat, 11 Jan 2020 08:00:47","[cat, cats, lol, lolcat, lolcats]",jpg
7,190190785530,https://66.media.tumblr.com/522386ce0644901d60...,"Sat, 11 Jan 2020 06:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
8,190174143051,https://66.media.tumblr.com/5f1a4eec3910ec2262...,"Fri, 10 Jan 2020 08:00:54","[cat, cats, lol, lolcat, lolcats]",jpg
9,190153974947,https://66.media.tumblr.com/e70ac81ae13bf839f8...,"Thu, 09 Jan 2020 06:01:10","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [296]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [297]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [ ]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [299]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [310]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [301]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [302]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [303]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [304]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [305]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [306]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [307]:
pdf_url="https://www.mdrc.org/sites/default/files/Understanding_Poverty_Lit_Review.pdf"
request = requests.get(pdf_url, stream=True)

In [385]:
text = []
mapping = {'overview': [3530,4320], 
           'purpose': [4330,4772],
           'research_question': [4800,5202],
           'key_findings': [5218,6928],
           'methods':[6935,8000]}

for key in mapping.keys():
    start, end = mapping[key]
    para = readPDF(io.BytesIO(request.content))[start:end].strip()
    para = re.sub(r'(-)*\n','',para)
    para = re.sub(r'iii \x0c ','',para)
    text.append(para)

In [390]:
df3 = pandas.DataFrame({'sections': list(mapping.keys()), 'text': text})
df3

,sections,text
0,overview,One in five American children — 14.5 million —...
1,purpose,The purpose of this review is to summarize the...
2,research_question,This review summarizes the qualitative literat...
3,key_findings,• Material deprivation is a fundamental aspec...
4,methods,A systematic approach was taken to revie...


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

